In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Dropout, Conv2D, MaxPooling2D, Input
from keras.layers.normalization import BatchNormalization
from utils import utils

In [ ]:
X_train, y_train = utils.get_train('data/IMG/', 'data/driving_log.csv')

In [ ]:
X_train.shape, y_train.shape

In [ ]:
img_height, img_width = X_train.shape[1], X_train.shape[2]
img_shape = X_train.shape[1:]

In [ ]:
def create_model(p=0.6, input_shape=(160, 320, 3)):
    model = Sequential()
    model.add(Lambda(lambda x: x / 255. - .5, input_shape=input_shape))
    # Convolutional 
    model.add(Conv2D(32, (3, 3), padding='same', activation ='relu'))
    model.add(MaxPooling2D(pool_size =(2, 2)))
    model.add(Conv2D(64, (3, 3), padding='same', activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Flatten
    model.add(Flatten())
    # Fully-Connection
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(p/2))
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(p))
    model.add(Dense(1))
    
    model.compile(lr=1e-3, loss='mse', optimizer='adam', metrics=['accuracy'])   
    return model 

In [ ]:
def train(X, y, epochs, batch_size):
    model = create_model(0.6, img_shape)
    history = model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2, shuffle=True)
    model.save('models/model.h5')
    return history

In [ ]:
hist = train(X_train, y_train, epochs=5, batch_size=32)